In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, LinearRegression,LassoCV
from sklearn.metrics import mean_squared_error
#from sklearn.cross_validation import KFold
plt.style.use('ggplot')
%matplotlib inline

In [27]:
df=pd.read_csv(r"C:\nydsa bootcamp slides\Projects\3\git\machine-learning_house-prices\Chaoran\AmesDummiesOrdinal.csv",
               index_col=0)
#SalePrice as output, all other variables as input
SalePrice=df.SalePrice
lasso_df=df.drop('SalePrice',axis=1)

In [28]:
df.head()

,LotFrontage,LotArea,MasVnrArea,TotalBsmtSF,LowQualFinSF,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,...,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MoSold_Autumn,MoSold_Spring,MoSold_Winter
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,196.0,856,0,3,1,8,0,2,...,1,3,4,4,3,1,1,0,0,1
2,80.0,9600,0.0,1262,0,3,1,6,1,2,...,4,3,4,4,3,1,1,0,1,0
3,68.0,11250,162.0,920,0,3,1,6,1,2,...,4,3,4,4,3,1,1,1,0,0
4,60.0,9550,0.0,756,0,3,1,7,1,3,...,5,2,4,4,3,1,1,0,0,1
5,84.0,14260,350.0,1145,0,4,1,9,1,3,...,4,3,4,4,3,1,1,0,0,1


### Normalize numeric columns

In [29]:
#check maximum value of each feature to decide which columns we want to normalize
list(map(lambda x:print(x,max(lasso_df[x])),lasso_df))

LotFrontage 313.0
LotArea 215245
MasVnrArea 1600.0
TotalBsmtSF 3206
LowQualFinSF 572
BedroomAbvGr 8
KitchenAbvGr 3
TotRmsAbvGrd 14
Fireplaces 3
GarageCars 4
GarageArea 1390
WoodDeckSF 857
OpenPorchSF 547
EnclosedPorch 552
3SsnPorch 508
ScreenPorch 480
PoolArea 738
MiscVal 15500
BsmtScore 5.997715851987209
TotalSF 4476
TotalBath 6.0
MSSubClass_30 1
MSSubClass_40 1
MSSubClass_45 1
MSSubClass_50 1
MSSubClass_60 1
MSSubClass_70 1
MSSubClass_75 1
MSSubClass_80 1
MSSubClass_85 1
MSSubClass_90 1
MSSubClass_120 1
MSSubClass_160 1
MSSubClass_180 1
MSSubClass_190 1
MSZoning_C (all) 1
MSZoning_FV 1
MSZoning_RH 1
MSZoning_RM 1
Street_Grvl 1
Alley_Grvl 1
Alley_Pave 1
LandContour_Bnk 1
LandContour_HLS 1
LandContour_Low 1
LotConfig_Corner 1
LotConfig_CulDSac 1
LotConfig_FR2 1
LotConfig_FR3 1
LandSlope_Mod 1
LandSlope_Sev 1
Neighborhood_Blmngtn 1
Neighborhood_Blueste 1
Neighborhood_BrDale 1
Neighborhood_BrkSide 1
Neighborhood_ClearCr 1
Neighborhood_CollgCr 1
Neighborhood_Crawfor 1
Neighborhood_Edwards

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
# At the begining i tried to use normalize directly on all cols, then noticed it will then normalize based on all datain 
# the 2-d array. Then the large value columns such as LotArea will make other cols to a really small value. 
#Therefore change the idear to standardize values to 0-1 for each columns 

'''
from sklearn import preprocessing
norm_df=lasso_df[['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','LowQualFinSF','GarageArea','WoodDeckSF','OpenPorchSF',
                 'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','TotalSF','GarageAge']]

normalize=preprocessing.normalize(norm_df.values,norm='l1')
norm2_df=pd.DataFrame(normalize,columns=norm_df.columns,index=norm_df.index)

lasso_df[['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','LowQualFinSF','GarageArea','WoodDeckSF','OpenPorchSF',
                 'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','TotalSF','GarageAge']]=norm2_df '''

In [23]:
# Standarize column with maximum values larger than 100 
# ['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','LowQualFinSF','GarageArea','WoodDeckSF','OpenPorchSF',
#       'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','TotalSF','YearsAgoBuilt','GarageAge']
 
from sklearn import preprocessing

'''The function to standardize the columns seperately. 
df is the dataframe needs the standardization.  
collist is the list of columns which need to be standardized'''

def standize_df(df,collist):
    for col in collist:
        stan_df=df[[col]]
        min_max_scaler = preprocessing.MinMaxScaler()
        standardized=min_max_scaler.fit_transform(stan_df)
        stan2_df=pd.DataFrame(standardized,columns=stan_df.columns,index=stan_df.index)
        df[[col]]=stan2_df
    return df


In [32]:
collist=['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','LowQualFinSF','GarageArea','WoodDeckSF','OpenPorchSF',
       'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','TotalSF','YearsAgoBuilt','GarageAge']
lasso_df=standize_df(lasso_df,collist)
lasso_df.tail()

,LotFrontage,LotArea,MasVnrArea,TotalBsmtSF,LowQualFinSF,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,...,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MoSold_Autumn,MoSold_Spring,MoSold_Winter
Id,,,,,,,,,,,,,,,,,,,,,
1456,0.140411,0.030929,0.000000,0.297255,0.0,3,1,7,1,2,...,4,3,4,4,3,1,1,0,0,0
1457,0.219178,0.055505,0.074375,0.480973,0.0,3,1,7,2,2,...,4,2,4,4,3,1,4,0,0,1
1458,0.154110,0.036187,0.000000,0.359326,0.0,4,1,9,2,1,...,5,3,4,4,3,1,5,0,1,0
1459,0.160959,0.039342,0.000000,0.336245,0.0,2,1,5,0,1,...,1,2,4,4,3,1,1,0,1,0
1460,0.184932,0.040370,0.000000,0.391765,0.0,3,1,6,0,1,...,1,4,4,4,3,1,1,0,0,0


### After normalization, we can start to build up the lasso model
* first split the dataset to 80% - 20% training and test dataset
* Then find the best lambda
* Check coef=0 features

In [34]:
# Split the Ames_df into a 80% train-set and a 20% test set. 
# I tested a few seed set, the seed19 gives the best result when consider about all (alpha value, score, RMSE) together
# with differet seed (different data split way) the lambda will change , and the result will be slightly different
# I think this is due to some outlier nodes of some features. Once we clean the data set better we should get more stable result

np.random.seed(19)
testIdxes = np.random.choice(range(1458), size= 292, replace=False)
trainIdxes = list(set(range(1458))-set(testIdxes))
trainX = lasso_df.iloc[trainIdxes]
testX  = lasso_df.iloc[testIdxes]
trainY = SalePrice.iloc[trainIdxes]
testY  = SalePrice.iloc[testIdxes]

# Use np.linspace to generate 40 alpha between 0.001 and 200
alphaRange = np.linspace(1e-3,200,40) 

### Start with finding the best Lambda , still use cross validation

In [35]:
lasso_cv=LassoCV(alphas=alphaRange,cv=10)
lasso_cv.fit(trainX,trainY)
lasso_best_alpha=round(lasso_cv.alpha_,3)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [36]:
lasso_best_alpha

51.283

In [37]:
lasso_cv.score(trainX,trainY)

0.9188862527807476

In [38]:
# build up lasso model with the best alpha we get above
lasso=Lasso(alpha=lasso_best_alpha)
lasso.fit(trainX,trainY)

Lasso(alpha=51.283, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [39]:
lasso_inte=lasso.intercept_
lasso_inte

-122994.6157090844

In [40]:
lasso_coef=pd.Series(lasso.coef_, index=lasso_df.columns)
lasso_coef_sorted=np.abs(lasso_coef).sort_values(ascending=False)
zero_coef=lasso_coef_sorted[lasso_coef_sorted.values==0]

In [41]:
len(zero_coef)
# here it shows 100 columns has coef value=0

76

In [42]:
lasso_predict=lasso.predict(testX)
RMSE=np.sqrt(mean_squared_error(testY,lasso_predict))
RMSE

22163.351116891194

In [43]:
PredictionsTest=lasso.predict(testX)
RMSLEvalue = np.sqrt(np.mean(np.power(np.log1p(testY)-np.log1p(PredictionsTest), 2)))
RMSLEvalue

0.1425193107138621

In [44]:
#save the list of coef = 0 to zero_coef_lasso.csv file
zero_coef.to_csv("zero_coef_lasso.csv")

In [45]:
#drop the coef=0 columns to get a sub df, check the shape to make sure the column num looks good
leftcol=df.drop(list(zero_coef.index),axis=1)

In [46]:
leftcol.shape

(1458, 110)

In [47]:
# Save the new df with no coef=0 columns to lasso.csv file
leftcol.to_csv("lasso.csv")

In [48]:
zero_coef

HouseStyle_1.5Unf        0.0
MSSubClass_80            0.0
MSSubClass_30            0.0
MSSubClass_40            0.0
MSSubClass_45            0.0
MSSubClass_50            0.0
Utilities_NoSeWa         0.0
MSSubClass_70            0.0
LotShape_IR3             0.0
MSSubClass_85            0.0
Condition_PosA           0.0
SaleCondition_Family     0.0
SaleCondition_Alloca     0.0
SaleCondition_AdjLand    0.0
MSSubClass_180           0.0
SaleType_Oth             0.0
MSZoning_C (all)         0.0
Condition_RRAn           0.0
Condition_RRNn           0.0
SaleType_ConLD           0.0
MiscVal                  0.0
PavedDrive               0.0
GarageAge                0.0
LowQualFinSF             0.0
Exterior_Other           0.0
Condition_RRNe           0.0
Exterior_AsphShn         0.0
Exterior_CBlock          0.0
Exterior_Stone           0.0
Exterior_BrkComm         0.0
                        ... 
RoofStyle_Mansard        0.0
Neighborhood_OldTown     0.0
GarageType_CarPort       0.0
Neighborhood_S